通过和风天气API 查询 （https://console.qweather.com/#/apps）

In [9]:
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

QWEATHER_API_KEY = "<API_KEY>"
LOCATION_QUERY_URL = "https://geoapi.qweather.com/v2/city/lookup?"
WEATHER_QUERY_URL = "https://devapi.qweather.com/v7/weather/now?"

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def query_qweather_request(city):
    headers = { "Content-Type": "application/json"}

    try:
        resp = requests.get(LOCATION_QUERY_URL + "location=" + city + "&key=" + QWEATHER_API_KEY).json()
    except Exception as e:
        print("Unable to query GEO location")
        print(f"Exception: {e}")
        return None
    
    if resp['code'] != '200' or len(resp['location']) < 1:
        print("Failed to retrieve location code! Resp:" + str(resp))
        return None
    
    city_code = resp['location'][0]['id']
    
    try:
        weather_response = requests.get(WEATHER_QUERY_URL + "location=" + city_code + "&key=" + QWEATHER_API_KEY).json()
    except Exception as e:
        print("Unable to query weather")
        print(f"Exception: {e}")
        return None
    
    if weather_response['code'] != '200' or weather_response['now'] is None:
        print("Failed to retrieve weather! Resp:" + str(weather_response))
        return None
    
    return "气温:" + weather_response['now']["temp"] + " 天气:" + weather_response['now']["text"] + " 风向:" + weather_response['now']["windDir"]



resp = query_qweather_request("shanghai, China")
print(resp)



气温:24 天气:多云 风向:东北风


In [10]:
GPT_MODEL = "gpt-3.5-turbo"
OPENAI_BASE_URL = "<OPEN_AI URL>"
OPENAI_API_KEY = "<API_KEY>"

In [11]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def openai_chat_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = { "Content-Type": "application/json", "Authorization": "Bearer " + OPENAI_API_KEY }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            OPENAI_BASE_URL + "/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "yellow",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [12]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {  
                "location": {  
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                }
            },
            "required": ["location"]
        }
    }
]

In [13]:
def execute_function_call(message):
    """执行函数调用"""
    # 判断功能调用的名称是否为 "ask_database"
    if message["function_call"]["name"] == "get_current_weather":
        # 如果是，则获取功能调用的参数，这里是 SQL 查询
        location = json.loads(message["function_call"]["arguments"])["location"]
        # 使用 ask_database 函数执行查询，并获取结果
        results = query_qweather_request(location)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results  # 返回结果

In [14]:
messages = []

messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})

messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "What's the weather like today"  # 用户询问今天的天气情况
})

chat_response = openai_chat_request(
    messages, functions=functions
)
messages.append(chat_response.json()["choices"][0]["message"])

pretty_print_conversation(messages)


system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, I can help you with that. Could you please provide me with the city and state you'd like to know the weather for?



In [16]:
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "Hangzhou"  # 用户的消息内容
})

# 再次使用定义的chat_completion_request函数发起一个请求，传入更新后的messages和functions作为参数
chat_response = openai_chat_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的新的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant[content]: Sure, I can help you with that. Could you please provide me with the city and state you'd like to know the weather for?

user: Shanghai, China.

function (get_current_weather): 气温:24 天气:多云 风向:东北风

user: Hangzhou

function (get_current_weather): 气温:23 天气:多云 风向:南风

